In [1]:
# Detect current folder to avoid package import error
import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

## Step 1: Pre-process Geo Data (Grid Connections Excluded)

Pre-process to get the Set-up Dictionary for building up **MPDP** frame model which calculates expressions like distances between nodes and EESC, etc. 

In [2]:
# To import Opensource packages
import numpy as np
import pandas as pd
#import geopandas as gpd 
#from shapely.geometry import Point, MultiPoint
#import time
#import pickle
#from datetime import datetime

import idaes
import numpy as np
import pyomo.environ as pyo

from pyomo.core.util import quicksum 
from pyomo.core.expr.current import evaluate_expression 

ModuleNotFoundError: No module named 'idaes'

In [ ]:
m_name = 'test_model'
model = pyo.ConcreteModel(name = m_name )
print(f"Built an empty concrete pyomo model named {model.name}. \nDefining sets   ...") 

## Setup Sets

In [ ]:
model.I_update = pyo.Set(
    initialize = [1, 2, 3],
    # initialize = setup_dict['model_sets']['I_update'], 
    # domain = pyo.NonNegativeIntegers,
    doc = 'Candidate locations with EXISTING CSs and CPs.'
)


model.I_newBuild = pyo.Set(
    initialize = [4,5,6],
    # domain = pyo.NonNegativeIntegers,  
    doc = 'Candidate locations for NEW CS.'
) 


model.J = pyo.Set(
    initialize = ['a', 'b', 'c'],
    # domain = pyo.NonNegativeIntegers ,    
    doc = 'Set of CD centers.'
    )


model.I = model.I_update | model.I_newBuild # All candidate Locs

model.T = pyo.Set(
    initialize = ['spring', 'winter'],
    # domain = pyo.NonNegativeIntegers ,    
    doc = 'Set of time periods.'
    )

In [ ]:
model.display()

In [ ]:
model.I.display()

## Setup Parameters

In [ ]:
# model.nodes = pyo.Param(
#     (model.I | model.J), 
#     initialize = [(1,2), (3,4), (4,5)],
#     within = pyo.Any,
#     doc = """Coordinates of all nodes including candilocs and CD centers, used for calculating walking distances"""
# )

# model.N_newCPs = pyo.Param(
#     initialize = setup_dict['params_basic']['N_newCPs'],
#     domain = pyo.NonNegativeIntegers,
#     mutable = True, 
#     # , default = 14
#     doc = 'Maximum total amount of new CPs the investors want to install.'
# )

## Setup DVs

In [ ]:
model.x = pyo.Var(
    model.I, 
    domain = pyo.NonNegativeReals, 
    initialize = 0.50,
    bounds = (0.20, 1),
    doc = '''This is the charging price for each CS i \in I'''
)

In [ ]:
model.x.display()

## Setup Objective Function

In [ ]:
model.gross_revenue = quicksum(
        model.calD[i,j,t] * model.tau[i,j] * model.calA[j,t] \
            * model.z[i,j,t] * model.u[i] \
            for i in model.I for j in model.J for t in model.T) * 365

model.cs_placement_cost = quicksum(
    model.c_x[i] * model.x[i] + model.c_y * model.y[i] \
        for i in model.I
)

model.obj_profit_no_grid = pyo.Objective(
        rule = model.gross_revenue - model.cs_placement_cost , 
        sense = pyo.maximize
    )

## Example Price Optimization


In [ ]:
from pyomo.environ import *
import random

# Create a model
model = ConcreteModel()

Data_dict = {
    'sites': ["DE", "NO"],
    "time_periods": ["Q1", "Q2", "Q3", "Q4"],
    'total_cost': {
        ('DE', 'Q1'): 0.028,
        ('DE', 'Q2'): 0.76,
        ('DE', 'Q3'): 1.44,
        ('DE', 'Q4'): 0.98,
        ('NO', 'Q1'): 1.14,
        ('NO', 'Q2'): 0.43,
        ('NO', 'Q3'): 0.78,
        ('NO', 'Q4'): 0.32,
        },
    'variable_cost': {
        ('DE', 'Q1'): 0.028,
        ('DE', 'Q2'): 0.76,
        ('DE', 'Q3'): 1.44,
        ('DE', 'Q4'): 0.98,
        ('NO', 'Q1'): 1.14,
        ('NO', 'Q2'): 0.43,
        ('NO', 'Q3'): 0.78,
        ('NO', 'Q4'): 0.32,
        },
    'fixed_cost': {
        ('DE', 'Q1'): 0.028,
        ('DE', 'Q2'): 0.76,
        ('DE', 'Q3'): 1.44,
        ('DE', 'Q4'): 0.98,
        ('NO', 'Q1'): 1.14,
        ('NO', 'Q2'): 0.43,
        ('NO', 'Q3'): 0.78,
        ('NO', 'Q4'): 0.32,
        },
    'p_min': 0.1,
    'p_max': 0.5,
    'd_max': 100
}

# Define sets
model.S = Set(initialize=Data_dict['sites'])
model.T = Set(initialize=Data_dict['time_periods'])

# Define variables
model.d = Var(
    model.S, 
    model.T, 
    bounds=(0, Data_dict['d_max']),
    domain=NonNegativeReals,
    doc = 'dv: cd'
)

model.p = Var(
    model.S, 
    model.T,  
    bounds=(Data_dict['p_min'], Data_dict['p_max']),
    domain=NonNegativeReals,
    doc = 'dv: price'
)

# Define parameters
model.c = Param(
    model.S, 
    model.T, 
    initialize=Data_dict['total_cost'], 
    doc = 'param: cost'
)
# model.p_min = Param(initialize=p_min)
# model.p_max = Param(initialize=p_max)
# model.d_max = Param(initialize=d_max)

# Define objective
model.profit = Objective(
    expr = sum(
        model.d[s,t]*(model.p[s,t]-model.c[s,t]) \
            for s in model.S for t in model.T
        ), 
    sense = maximize
)

# Define constraints
# def price_constraints_rule(model, s, t):
#     return model.p_min <= model.p[s,t] <= model.p_max
# model.price_constraints = Constraint(model.S, model.T, rule=price_constraints_rule)

# def demand_constraints_rule(model, s):
#     return model.d[s,t] <= model.d_max
# model.demand_constraints = Constraint(model.S, rule=demand_constraints_rule)